In [23]:
#!/usr/bin/env python

try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2
import h5py

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
bpl.output_notebook()

     4037945 [pyplot.py:      switch_backend():211] [1480] Loaded backend Qt5Agg version unknown.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [2]:
fnames = ['/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/H10_M19_S59msCam9_substack_1.tif',
          '/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/H10_M19_S59msCam9_substack_2.tif'
         ]  # filename to be processed

In [3]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [4]:
# dataset dependent parameters
frate = 20                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries

mc_dict = {
    'fnames': fnames,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}

opts = params.CNMFParams(params_dict=mc_dict)

       62115 [params.py:                 set():780] [1480] Changing key fnames in group data from None to ['/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/H10_M19_S59msCam9_substack_1.tif', '/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/H10_M19_S59msCam9_substack_2.tif']
       62116 [params.py:                 set():780] [1480] Changing key fr in group data from 30 to 20
       62117 [params.py:                 set():780] [1480] Changing key max_shifts in group motion from (6, 6) to (5, 5)
       62118 [params.py:                 set():780] [1480] Changing key gSig_filt in group motion from None to (3, 3)
       62119 [params.py:                 set():780] [1480] Changing key strides in group motion from (96, 96) to (48, 48)
       62121 [params.py:                 set():780] [1480] Changing key overlaps in group motion from (32, 32) to (24, 24)


In [5]:
if motion_correct:
    # do motion correction rigid
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    fname_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                     np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)
        plt.subplot(1, 2, 1); plt.imshow(mc.total_template_rig)  # % plot template
        plt.subplot(1, 2, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
        plt.legend(['x shifts', 'y shifts'])
        plt.xlabel('frames')
        plt.ylabel('pixels')

    bord_px = 0 if border_nan is 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(fnames, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)

       69258 [motion_correction.py:motion_correct_rigid():258] [1480] Entering Rigid Motion Correction
       69259 [motion_correction.py:motion_correct_rigid():259] [1480] 1.3795735
       69462 [movies.py:      extract_shifts():256] [1480] min_val in extract_shifts: -0.297609384059906
/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/caiman/base/movies.py:258: UserWarning: ** Pixel averages are too negative. Removing 1 percentile. **
  '** Pixel averages are too negative. Removing 1 percentile. **')
/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/caiman/base/movies.py:277: UserWarning: Pixel averages are too negative for template. Removing 1 percentile.
  'Pixel averages are too negative for template. Removing 1 percentile.')
       69628 [movies.py:        apply_shifts():352] [1480] cubic interpolation
       69834 [movies.py:      extract_shifts():256] [1480] min_val in extract_shifts: -0.297609384059906
       70012 [movies.py:        ap

      114323 [mmapping.py:        save_portion():264] [1480] Index start and end are 21654 and 25263
      114335 [mmapping.py:        save_portion():282] [1480] done
      114336 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      114337 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      114341 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      114344 [mmapping.py:        save_portion():264] [1480] Index start and end are 25263 and 28872
      114358 [mmapping.py:        save_portion():282] [1480] done
      114359 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      114361 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passp

      115140 [mmapping.py:        save_portion():264] [1480] Index start and end are 68571 and 72180
      115153 [mmapping.py:        save_portion():282] [1480] done
      115154 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      115155 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      115159 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      115162 [mmapping.py:        save_portion():264] [1480] Index start and end are 72180 and 75789
      115453 [mmapping.py:        save_portion():282] [1480] done
      115455 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      115456 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passp

      115976 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      115980 [mmapping.py:        save_portion():264] [1480] Index start and end are 115488 and 119097
      116258 [mmapping.py:        save_portion():282] [1480] done
      116260 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      116261 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      116265 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      116268 [mmapping.py:        save_portion():264] [1480] Index start and end are 119097 and 122706
      116280 [mmapping.py:        save_portion():282] [14

      116989 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      117013 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      117016 [mmapping.py:        save_portion():264] [1480] Index start and end are 162405 and 166014
      117031 [mmapping.py:        save_portion():282] [1480] done
      117032 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      117033 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      117067 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_7

      118227 [mmapping.py:        save_portion():282] [1480] done
      118229 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      118230 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      118253 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      118256 [mmapping.py:        save_portion():264] [1480] Index start and end are 209322 and 212931
      118270 [mmapping.py:        save_portion():282] [1480] done
      118272 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      118272 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
  

      119590 [mmapping.py:        save_portion():264] [1480] Index start and end are 252630 and 256239
      119609 [mmapping.py:        save_portion():282] [1480] done
      119611 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      119612 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      119639 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      119667 [mmapping.py:        save_portion():264] [1480] Index start and end are 256239 and 259848
      119943 [mmapping.py:        save_portion():282] [1480] done
      119945 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      119946 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_P

      121071 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      121099 [mmapping.py:        save_portion():264] [1480] Index start and end are 299547 and 303156
      121420 [mmapping.py:        save_portion():282] [1480] done
      121422 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      121423 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      121448 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      121458 [mmapping.py:        save_portion():264] [1480] Index start and end are 303156 and 306765
      121475 [mmapping.py:        save_portion():282] [14

      122896 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      122923 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      122932 [mmapping.py:        save_portion():264] [1480] Index start and end are 346464 and 350073
      122947 [mmapping.py:        save_portion():282] [1480] done
      122949 [mmapping.py:        save_portion():256] [1480] Shape of Yr_tot is (3609, 202)
      122950 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0000_d1_480_d2_752_d3_1_order_C_frames_101_.mmap
      122978 [mmapping.py:        save_portion():258] [1480] Saving portion to /Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/memmap_0001_d1_480_d2_7

In [6]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [7]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thresh = .7   # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .8       # min peak value from correlation image
min_pnr = 10        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thresh': merge_thresh,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

      209937 [params.py:                 set():780] [1480] Changing key rf in group patch from None to 40
      209938 [params.py:                 set():780] [1480] Changing key stride in group patch from None to 20
      209939 [params.py:                 set():780] [1480] Changing key nb_patch in group patch from 1 to 0
      209940 [params.py:                 set():780] [1480] Changing key low_rank_background in group patch from True to None
      209941 [params.py:                 set():780] [1480] Changing key del_duplicates in group patch from False to True
      209943 [params.py:                 set():780] [1480] Changing key p in group preprocess from 2 to 1
      209944 [params.py:                 set():780] [1480] Changing key method_init in group init from greedy_roi to corr_pnr
      209946 [params.py:                 set():780] [1480] Changing key K in group init from 30 to None
      209947 [params.py:                 set():780] [1480] Changing key gSig in group init fro

In [8]:
# compute some summary images (correlation and peak to noise)
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters
inspect_correlation_pnr(cn_filter, pnr)

      250276 [colorbar.py:_get_ticker_locator_formatter():507] [1480] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x10f707358>
      250278 [colorbar.py:        update_ticks():536] [1480] Using auto colorbar locator on colorbar
      250279 [colorbar.py:        update_ticks():537] [1480] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x10f707358>
      250279 [colorbar.py:         _add_solids():693] [1480] Setting pcolormesh
      250343 [colorbar.py:_get_ticker_locator_formatter():507] [1480] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x10f789128>
      250344 [colorbar.py:        update_ticks():536] [1480] Using auto colorbar locator on colorbar
      250345 [colorbar.py:        update_ticks():537] [1480] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x10f789128>
      250346 [colorbar.py:         _add_solids():693] [1480] Setting pcolormesh
      250483 [_base.py:_update_title_position():2515] [1480] update_title_pos
  

In [9]:
# print parameters set above, modify them if necessary based on summary images
print(min_corr) # min correlation of peak (from correlation image)
print(min_pnr)  # min peak to noise ratio

0.8
10


In [10]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)

In [11]:
cnm.fit(images)

/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
      370796 [params.py:                 set():780] [1480] Changing key init_batch in group online from 200 to 202
      370796 [cnmf.py:                 fit():404] [1480] (202, 480, 752)
      370797 [params.py:                 set():780] [1480] Changing key medw in group spatial from None to (3, 3)
      370798 [params.py:                 set():780] [1480] Changing key se in group spatial from None to [[1 1 1]
 [1 1 1]
 [1 1 1]]
      370800 [params.py:                 set():780] [1480] Changing key ss in group spatial from None to [[1 1 1]
 [1 1 1]
 [1 1 1]]
      370801 [cn

(80, 80)
228.8774380683899
Transforming patches into full matrix
Skipped %d Empty Patch 0
Generating background
Generating background DONE
merging


/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/caiman/source_extraction/cnmf/merging.py:141: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  C_corr[ii, overlap_indeces] = corr_values


[8 9]
[3 7]


/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/scipy/signal/spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 202, using nperseg = 202
  .format(nperseg, input_length))
/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/caiman/source_extraction/cnmf/deconvolution.py:991: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]


No neurons merged!


In [12]:
#%% plot contours of found components
Cn = cm.local_correlations(images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn)

In [13]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 3            # adaptive way to set threshold on the transient size
r_values_min = 0.85    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

     1498365 [params.py:                 set():780] [1480] Changing key min_SNR in group quality from 2.5 to 3
     1498366 [params.py:                 set():780] [1480] Changing key rval_thr in group quality from 0.8 to 0.85
     1498367 [params.py:                 set():780] [1480] Changing key use_cnn in group quality from True to False
     1498373 [components_evaluation.py:estimate_components_quality():682] [1480] EVALUATING IN PARALLEL... NOT RETURNING ERFCs


 ***** 
Number of total components:  13
Number of accepted components:  13


In [14]:
np.shape(cnm.estimates.C)

(13, 202)

In [24]:
hf = h5py.File('/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/output_data.h5', 'w')

In [38]:
cnm.estimates.A.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
hf.create_dataset('A', data=cnm.estimates.A.toarray())
hf.create_dataset('C', data=cnm.estimates.C)


<HDF5 dataset "C": shape (13, 202), type "<f8">

In [40]:
hf.close()

In [16]:
for i in range(len(cnm.estimates.C)):
    plt.plot(cnm.estimates.C[i])
plt.show()

     1711029 [_base.py:_update_title_position():2515] [1480] update_title_pos


In [17]:
# without background
cnm.estimates.play_movie(images, q_max=99.9, magnification=2,
                                 include_bck=False, gain_res=4, bpx=bord_px)

In [18]:
cnm.save('/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/cmn_output.hdf5')

     1771749 [utils.py:recursively_save_dict_contents_to_group():440] [1480] dims is a tuple ****
/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
     1771765 [utils.py:recursively_save_dict_contents_to_group():440] [1480] dims is a tuple ****
     1771771 [utils.py:recursively_save_dict_contents_to_group():440] [1480] dxy is a tuple ****
     1771818 [utils.py:recursively_save_dict_contents_to_group():440] [1480] gSig is a tuple ****


ValueError: Cannot save <class 'tuple'> type.